In [39]:
# Imports
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Update with your API key
client = OpenAI(api_key=os.environ.get("API_KEY"))

# Open the CSV file in "read binary" (rb) mode, with the "assistants" purpose
file = client.files.create(
  file=open("Intro-to-Python-Part-1.pdf", "rb"),
  purpose='assistants'
)

# Create and configure the assistant
assistantResumer = client.beta.assistants.create(
    name="Teacher",
    instructions="""Please take the lecture PDFs and create detailed keypoints that are longer and more informative. The summary should include:
- All major topics covered in the lecture, with a concise explanation of each.
- Key concepts and definitions should be clearly stated and elaborated upon to ensure thorough understanding.
- Any complex academic content must be broken down into simpler terms for better grasp, while preserving the necessary technical terms to accurately convey the concepts.
- Examples or analogies that make the information relatable and easier to understand for an audience with varying levels of knowledge.
- Questions that commonly arise from the material, with straightforward answers to them.
- Maintain a warm and conversational tone throughout the text to keep it engaging.
The goal is to provide a full but accessible overview of the lecture content that is not just bullet points, but a rich narrative that educates and engages readers of all levels. Please ensure the summary is comprehensive and extends over several paragraphs, effectively translating dense material into an easy-to-follow format.""",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

# Create a thread where the conversation will happen
thread = client.beta.threads.create()

# Create the user message and add it to the thread
assistantResumermessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="File:",
)

# Create the Run, passing in the thread and the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantResumer.id
)

# Periodically retrieve the Run to check status and see if it has completed
# Should print "in_progress" several times before completing
while run.status != "completed":
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    if keep_retrieving_run.status == "completed":
        break

# Retrieve messages added by the Assistant to the thread
assistantResumerAllMessages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# # Print the messages from the user and the assistant
# print("###################################################### \n")
# print(f"USER: {message.content[0].text.value}")
print(f"ASSISTANT: {assistantResumerAllMessages.data[0].content[0].text.value}")

ASSISTANT: Welcome to the world of Python, the versatile and widely-used programming language that's particularly favored in scientific, academic, and web development fields due to its readability and efficiency. This Introduction to Python, part one of the tutorial, is designed to help you begin your journey with this powerful tool, teach you the core concepts, and acquaint you with the Python way of programming.

### What's Inside Python?

Presenting Python as a high-level, interpreted language, this tutorial covers the essentials that make Python suitable for a wide array of tasks:

- **General Purpose**: Python is not limited to one kind of application. From web frameworks to data analysis, Python's design makes it a fit for various tasks.
  
- **Multiple Programming Paradigms**: Python shines with its support for structured (procedural) and object-oriented programming, making it adaptable to different problems.

- **Automatic Memory Management**: Python handles the allocation and 

In [40]:
assistantQuizMaster = client.beta.assistants.create(
    name="Teacher",
    instructions="""You are tasked with crafting questions designed to aid students' learning and retention based on what they give you. You should generate a balanced mix of multiple-choice and short-answer questions. Each question should be concise, clear, and directly related to the provided content. Present your questions in JSON format within its category and ensure the 'Answer' field is filled with the correct answer. Focus on maintaining a user-centered approach by adapting the question difficulty to individual study needs and promoting engagement.
### Example formating:
{
  "multiple_choice": [
    {
      "type": "multiple_choice",
      "question": "Who created the Python programming language?",
      "options": [
        "a) Linus Torvalds",
        "b) James Gosling",
        "c) Guido Van Rossum",
        "d) Dennis Ritchie"
      ],
      "answer": "c) Guido Van Rossum"
    }
  ],
  "short_answer": [
    {
      "type": "short_answer",
      "question": "What is the main advantage of Python being an interpreted language?",
      "answer": "The main advantage is the faster development cycle since there's no need for a separate compilation step, making it ideal for rapid prototyping and iterative development."
    }
  ]
}
""",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
)

# Create a thread where the conversation will happen
thread = client.beta.threads.create()

# Create the user message and add it to the thread
assistantQuizMastermessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f'{assistantResumerAllMessages.data[0].content[0].text.value}',
)

# Create the Run, passing in the thread and the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantQuizMaster.id
)

# Periodically retrieve the Run to check status and see if it has completed
# Should print "in_progress" several times before completing
while run.status != "completed":
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    if keep_retrieving_run.status == "completed":
        break

# Retrieve messages added by the Assistant to the thread
assistantQuizMasterAllMessages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [41]:
print(assistantQuizMasterAllMessages.data[0].content[0].text.value)

```json
{
  "multiple_choice": [
    {
      "type": "multiple_choice",
      "question": "Which programming paradigms does Python support?",
      "options": [
        "a) Object-oriented programming",
        "b) Structured (procedural) programming",
        "c) Functional programming",
        "d) Both a and b"
      ],
      "answer": "d) Both a and b"
    },
    {
      "type": "multiple_choice",
      "question": "What is the recommended version of Python for beginners, according to the tutorial?",
      "options": [
        "a) Python 2",
        "b) Python 3",
        "c) Either, as differences are negligible for beginners",
        "d) Neither, beginners should start with a different programming language"
      ],
      "answer": "b) Python 3"
    },
    {
      "type": "multiple_choice",
      "question": "What environment is highlighted as key for Python programming in the tutorial?",
      "options": [
        "a) Jupyter Notebook",
        "b) Microsoft Visual Studio",
   

In [42]:
with open('questions.json', 'w') as f:
    f.write(all_messages.data[0].content[0].text.value)